In [1]:
# -*- coding: utf-8 -*-
import asyncio
import websockets
import json
import logging
import base64
import time
import sys
from typing import Set, Optional, Coroutine, Any

try:
    from jetbot import Robot, Camera, bgr8_to_jpeg
except ImportError as e:
    print(f"Error importing jetbot library: {e}. Ensure installed for Python {sys.version_info.major}.{sys.version_info.minor}")
    # JetBot library might have compatibility issues below certain Python versions or require specific setup.
    # If the script continues, robot and camera will be None.
    Robot = None
    Camera = None
    bgr8_to_jpeg = None
    # sys.exit(1) # Removed exit to allow observation if script runs without hardware

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("JetBotDriver")

WEBSOCKET_HOST = "0.0.0.0"
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300
FPS = 1.0
FRAME_SEND_INTERVAL = 1.0 / FPS

robot: Optional[Robot] = None
camera: Optional[Camera] = None
hardware_initialized = False

# --- Hardware Initialization ---
try:
    # Only initialize if Robot and Camera were successfully imported
    if Robot and Camera:
        robot = Robot()
        # Try different camera instantiation methods if default fails
        try:
            camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
        except TypeError: # Handle potential older jetbot lib versions
             logger.warning("Camera.instance() failed with TypeError, trying legacy method.")
             camera = Camera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT) # Or Camera() if default size needed

        if robot is None or camera is None:
             raise RuntimeError("Robot or Camera object is None after instantiation.")

        logger.info("JetBot hardware initialized.")
        # Check camera value after a short delay
        time.sleep(1.5) # Allow camera warmup
        if camera.value is None:
            logger.warning("Camera init OK, but initial frame is None. Check hardware connection or driver.")
        hardware_initialized = True
    else:
         logger.error("JetBot libraries not fully imported. Hardware will not be used.")

except FileNotFoundError:
    logger.error("JetBot init failed: Motor driver files not found. Check I2C setup.")
except Exception as e:
    logger.error(f"JetBot hardware initialization failed: {e}", exc_info=True)

# --- Global State ---
connected_clients: Set[websockets.WebSocketServerProtocol] = set()
main_server_task: Optional[asyncio.Task] = None


# --- Command Handling ---
async def handle_command(command: str):
    if not hardware_initialized or not robot:
        # logger.warning("Hardware not available, command ignored.")
        return

    # Define speeds (ensure these work for your robot)
    speeds = {
        "forward_slow":   (0.25, 0.25), "forward_medium": (0.40, 0.40), "forward_fast":   (0.60, 0.60),
        "backward_slow":  (-0.25, -0.25),"backward_medium":(-0.40, -0.40),"backward_fast":  (-0.60, -0.60),
        "left_slow":      (0.0, 0.25),    "left_medium":    (0.0, 0.40),   "left_fast":      (0.0, 0.60),
        "right_slow":     (0.25, 0.0),    "right_medium":   (0.40, 0.0),   "right_fast":     (0.60, 0.0),
        "stop":           (0.0, 0.0)
    }
    left_speed, right_speed = speeds.get(command, (0.0, 0.0))

    try:
        robot.set_motors(left_speed, right_speed)
    except Exception as e:
        logger.error(f"Error setting motors for command '{command}': {e}")
        if robot:
            try: robot.stop()
            except Exception as stop_e: logger.error(f"CRITICAL: Failed to stop motors after error: {stop_e}")

# --- Camera Frame Sending ---
async def send_camera_frames(websocket: websockets.WebSocketServerProtocol):
    if not hardware_initialized or not camera or not bgr8_to_jpeg:
        logger.error("Camera/Conversion function not available, cannot send frames.")
        # Send an error message? Or just close gracefully?
        try:
            await websocket.close(code=1011, reason="Camera hardware unavailable") # Internal error code
        except Exception:
            pass # Ignore close errors if connection already gone
        return

    loop = asyncio.get_event_loop()
    last_frame_time = loop.time()

    while websocket.open:
        current_time = loop.time()
        wait_time = FRAME_SEND_INTERVAL - (current_time - last_frame_time)
        if wait_time > 0:
            await asyncio.sleep(wait_time)
        last_frame_time = loop.time() # Update time *after* potential sleep

        try:
            frame = camera.value
            if frame is None:
                # logger.debug("Captured None frame") # Reduce log noise
                await asyncio.sleep(0.1) # Wait briefly if frame is None
                continue

            # Ensure conversion function is available
            if not bgr8_to_jpeg:
                 logger.error("bgr8_to_jpeg function not available!")
                 break

            jpeg_data = bgr8_to_jpeg(frame)
            image_base64 = base64.b64encode(jpeg_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))

        except (websockets.exceptions.ConnectionClosed, websockets.exceptions.WebSocketException) as e:
            # logger.info(f"Connection closed during frame send: {e}") # Reduce noise
            break # Exit loop normally on closed connection
        except Exception as e:
            logger.error(f"Error processing/sending frame: {e}", exc_info=True)
            # Consider if break is needed based on error type
            await asyncio.sleep(0.5) # Wait a bit after an error

# --- WebSocket Connection Handler ---
async def websocket_handler(websocket: websockets.WebSocketServerProtocol, path: str):
    """Handles a single client connection."""
    client_ip = websocket.remote_address[0]
    logger.info(f"Client connected: {client_ip}")
    connected_clients.add(websocket)

    # Start task to send camera frames for this specific client
    frame_sender_task = asyncio.ensure_future(send_camera_frames(websocket))

    try:
        # Listen for incoming commands from this client
        async for message in websocket:
            try:
                data = json.loads(message)
                command = data.get("command")
                if command:
                    # logger.debug(f"Received command: {command}") # Reduce noise
                    await handle_command(command) # Execute the command
            except json.JSONDecodeError:
                logger.warning(f"Invalid JSON from {client_ip}: {message[:100]}...")
            except Exception as e:
                logger.error(f"Error handling message from {client_ip}: {e}", exc_info=True)

    except websockets.exceptions.ConnectionClosedOK:
        logger.info(f"Connection from {client_ip} closed normally.")
    except websockets.exceptions.ConnectionClosedError as e:
        logger.warning(f"Connection from {client_ip} closed with error: {e.code} {e.reason}")
    except Exception as e:
        logger.error(f"Unexpected error in WebSocket handler for {client_ip}: {e}", exc_info=True)
    finally:
        logger.info(f"Disconnecting client: {client_ip}")
        if websocket in connected_clients:
            connected_clients.remove(websocket)
        # Cancel the frame sending task when client disconnects
        if frame_sender_task and not frame_sender_task.done():
            frame_sender_task.cancel()
            # No need to await cancellation here, cleanup happens later
        # Stop motors only if this was the LAST client (optional, depends on desired behavior)
        # if not connected_clients and hardware_initialized and robot:
        if hardware_initialized and robot: # Stop motors whenever a client disconnects for safety
            logger.info("Client disconnected, stopping motors.")
            try: robot.stop()
            except Exception as e: logger.error(f"Failed to stop motors on disconnect: {e}")


# --- Main Server Start/Stop Logic ---
async def start_server() -> Coroutine[Any, Any, websockets.WebSocketServer]:
    """Starts the WebSocket server."""
    # Ensure initial motor stop only if hardware is ready
    if hardware_initialized and robot:
        logger.info("Stopping motors before starting server...")
        try: robot.stop()
        except Exception as e: logger.error(f"Failed to stop motors initially: {e}")

    logger.info(f"Starting JetBot WebSocket server on {WEBSOCKET_HOST}:{WEBSOCKET_PORT}")
    # websockets.serve returns an awaitable in older versions too
    server = await websockets.serve(
        websocket_handler,
        WEBSOCKET_HOST,
        WEBSOCKET_PORT,
        ping_interval=20,
        ping_timeout=20
    )
    logger.info("Server started and listening.")
    return server # Return the server object if needed, though not used here directly

async def run_main_server():
    """Keeps the server running and handles restart logic."""
    global main_server_task
    while True:
        try:
            # Start the server and wait for it to run
            # In Python 3.6, serve() starts listening but doesn't block forever.
            # We need to keep the task alive.
            server_instance = await start_server()

            # Keep the server running by awaiting indefinitely until an error occurs or stopped
            # This replaces the problematic asyncio.Future() inside the loop
            await asyncio.sleep(3600) # Sleep for a long time, or use another method to block

        except OSError as e:
            if "Address already in use" in str(e):
                 logger.error(f"Port {WEBSOCKET_PORT} is already in use. Check other processes.")
            else:
                 logger.error(f"Server OS error: {e}", exc_info=True)
            logger.info("Retrying server start in 10 seconds...")
            await asyncio.sleep(10)
        except Exception as e:
            logger.error(f"WebSocket server failed unexpectedly: {e}", exc_info=True)
            logger.info("Restarting server in 5 seconds...")
            await asyncio.sleep(5)
        finally:
            # Cleanup code if the server instance exists and needs closing
            # (websockets library usually handles this internally on errors)
            pass

# --- Graceful Shutdown ---
def shutdown(loop: asyncio.AbstractEventLoop):
    """Initiates graceful shutdown."""
    logger.info("Shutdown requested.")

    # Stop motors first if hardware available
    if hardware_initialized and robot:
        try:
            logger.info("Stopping motors during shutdown...")
            robot.stop()
        except Exception as e:
            logger.error(f"Error stopping motors during shutdown: {e}")

    # Cancel the main server task
    global main_server_task
    if main_server_task and not main_server_task.done():
        logger.info("Cancelling main server task...")
        main_server_task.cancel()

    # Find and cancel all other tasks (like websocket handlers and frame senders)
    tasks = [t for t in asyncio.Task.all_tasks(loop=loop) if t is not asyncio.Task.current_task(loop=loop)]
    if tasks:
        logger.info(f"Cancelling {len(tasks)} running tasks...")
        for task in tasks:
            task.cancel()
        # Wait for tasks to finish cancelling
        loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))

    logger.info("Stopping event loop.")
    loop.stop()


# --- Main Execution Block ---
if __name__ == "__main__":
    loop = asyncio.get_event_loop()

    # Add signal handlers for graceful shutdown (works on Linux/macOS)
    try:
        import signal
        for sig in (signal.SIGINT, signal.SIGTERM):
            loop.add_signal_handler(sig, shutdown, loop)
        logger.info("Signal handlers registered for SIGINT and SIGTERM.")
    except ImportError:
        logger.warning("Signal handlers not available on this platform (likely Windows). Use Ctrl+C.")
    except Exception as e:
        logger.error(f"Failed to register signal handlers: {e}")


    try:
        # Create and store the main server task
        main_server_task = loop.create_task(run_main_server())
        logger.info("Starting event loop...")
        # run_forever() is the correct way to run the loop until stop() is called
        loop.run_forever()

    except KeyboardInterrupt:
        logger.info("KeyboardInterrupt received directly in main block.")
        # Signal handler should ideally catch this, but handle here as fallback
        if not loop.is_running():
             logger.warning("Loop was not running when KeyboardInterrupt occurred.")
        elif not loop.is_closed():
             shutdown(loop) # Call the shutdown sequence
        else:
             logger.info("Loop already closed.")

    except Exception as e:
         logger.error(f"Unhandled exception in main execution: {e}", exc_info=True)
         # Attempt shutdown even on unexpected errors
         if loop.is_running() and not loop.is_closed():
             shutdown(loop)

    finally:
        # Final cleanup after loop has stopped
        if not loop.is_closed():
            logger.info("Closing event loop.")
            # Additional cleanup required before closing loop in 3.6
            loop.run_until_complete(loop.shutdown_asyncgens())
            loop.close()
        logger.info("JetBot Driver application finished.")

2025-03-28 08:32:15,909 - JetBotDriver - INFO - JetBot hardware initialized.
2025-03-28 08:32:17,420 - JetBotDriver - INFO - Signal handlers registered for SIGINT and SIGTERM.
2025-03-28 08:32:17,424 - JetBotDriver - INFO - Starting event loop...
2025-03-28 08:32:17,427 - JetBotDriver - ERROR - Unhandled exception in main execution: This event loop is already running
Traceback (most recent call last):
  File "<ipython-input-1-e15d89d3c91c>", line 289, in <module>
    loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 425, in run_forever
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
2025-03-28 08:32:17,433 - JetBotDriver - INFO - Shutdown requested.
2025-03-28 08:32:17,437 - JetBotDriver - INFO - Stopping motors during shutdown...
2025-03-28 08:32:17,439 - JetBotDriver - INFO - Cancelling main server task...
2025-03-28 08:32:17,441 - JetBotDriver - INFO - Cancelling 1 running tasks...
2025-03-28 08:

RuntimeError: This event loop is already running